In [7]:
from utils.data_directory_manager import DataDirectoryManager
from utils.img_diff import OutputAnalysis
from utils.rasterize import rasterize_label

In [8]:
base_dir = "/home/dsa/DSA/images_manual"
expected_file = "/home/dsa/DSA/vectors/Landsat_8_Derived_Outcrop_Dataset_2016.shp"

In [3]:
dm = DataDirectoryManager(base_dir)

In [ ]:
actual_file = ""